In [1]:
import pandas as pd
import numpy as np
import tempfile
import requests
import pyreadr

def load_freMTPL_mini() -> pd.DataFrame:
    commit = "c49cbbb37235fc49616cac8ccac32e1491cdc619"
    seed_no = 9000

    url = f"https://github.com/dutangc/CASdatasets/raw/{commit}/data/freMTPL2freq.rda"

    # download to a temporary file
    temp = tempfile.NamedTemporaryFile(delete=False)
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        temp.write(r.content)
    temp.close()

    # load the .rda file
    result = pyreadr.read_r(temp.name)
    df = result["freMTPL2freq"]

    # ClaimRate = ClaimNb / Exposure
    df["ClaimRate"] = df["ClaimNb"] / df["Exposure"]

    # cap ClaimRate at 99.9th percentile
    cap = df["ClaimRate"].quantile(0.999)
    df["ClaimRate"] = np.minimum(df["ClaimRate"], cap)

    # cap VehAge at 50
    df["VehAge"] = np.minimum(df["VehAge"], 50)

    # convert character columns to categorical
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype("category")

    # drop columns
    df = df.drop(columns=["IDpol", "Exposure", "ClaimNb", "Density", "Region"])

    # round ClaimRate and convert to integer
    df["ClaimRate"] = df["ClaimRate"].round().astype(int)

    # sample 25,000 rows with fixed seed
    df = df.sample(n=25000, random_state=seed_no)

    # return as a clean pandas DataFrame
    return df


In [6]:
commit = "c49cbbb37235fc49616cac8ccac32e1491cdc619"
seed_no = 9000

url = f"https://github.com/dutangc/CASdatasets/raw/{commit}/data/freMTPL2freq.rda"

temp = tempfile.NamedTemporaryFile(delete=False)
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    temp.write(r.content)
temp.close()

result = pyreadr.read_r(temp.name)
print(result.keys())
obj = result[list(result.keys())[0]]
print(type(obj))

PyreadrError: matrix, array or table object with more than one vector!

In [2]:
df = load_freMTPL_mini()

PyreadrError: matrix, array or table object with more than one vector!